In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector

In [1]:
database = "ofertas_de_empleo"
db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "Scarface",
                             database = database)

query = """
SELECT 
    o.empresa,
    -- Crear la columna salario_media combinando salario_desde y salario_hasta
    (o.salario_desde + o.salario_hasta) / 2 AS salario_media,
    
    -- Reemplazar los valores NaN en la columna 'variabe' con 0
    IFNULL(o.variabe, 0) AS variabe,
    
    -- Crear la columna 'variable_booleano'
    IF(o.variabe = 0, 0, 1) AS variable_booleano,
    
    -- Reemplazar los valores NaN en la columna 'vacaciones' por la moda
    IFNULL(o.vacaciones, (SELECT VACACIONES 
                          FROM ofertas 
                          GROUP BY VACACIONES 
                          ORDER BY COUNT(*) DESC 
                          LIMIT 1)) AS vacaciones,
    
    -- Tecnologías relacionadas con la oferta (pueden ser varias)
    GROUP_CONCAT(DISTINCT t.tecnologia) AS tecnologias,
    
    -- Habilidades relacionadas con la oferta (pueden ser varias)
    GROUP_CONCAT(DISTINCT h.habilidad) AS habilidades,
    
    -- Idiomas asociados con la oferta
    GROUP_CONCAT(DISTINCT i.idioma) AS idiomas,
    
    -- Niveles de los idiomas
    GROUP_CONCAT(DISTINCT i.nivel) AS niveles_idioma,
    
    -- Información sobre la ciudad (sin coordenadas ni comunidad autónoma)
    cci.pais AS ciudad_pais,
    cci.ciudad AS ciudad_coordenada
FROM 
    ofertas o
LEFT JOIN 
    tecnologias_relacion tr ON o.id_oferta = tr.id_oferta
LEFT JOIN 
    tecnologias t ON tr.tec_id = t.tec_id
LEFT JOIN 
    habilidades_relacion hr ON o.id_oferta = hr.id_oferta
LEFT JOIN 
    habilidades h ON hr.hab_id = h.hab_id
LEFT JOIN 
    idiomas i ON o.id_oferta = i.id_oferta
LEFT JOIN 
    ciudades ci ON o.id_oferta = ci.id_oferta
LEFT JOIN 
    ciudades_coordenadas cci ON ci.ciudad = cci.ciudad
GROUP BY 
    o.id_oferta;


"""

# Cargar los datos en un DataFrame
df = pd.read_sql(query, db)

print(df.head())  # Verificar los datos

NameError: name 'mysql' is not defined

In [ ]:
df.iloc[284]